In [2]:
# 1.실습데이터 만들기
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import re
from collections import Counter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 디바이스:", device)

# 간단한 영→한 문장 데이터 (원하면 더 추가해도 됨!)
data = [
    ("I am a student.", "나는 학생이다."),
    ("He plays football.", "그는 축구를 한다."),
    ("She likes coffee.", "그녀는 커피를 좋아한다."),
    ("We study English.", "우리는 영어를 공부한다."),
    ("They are friends.", "그들은 친구이다."),
    ("I like pizza.", "나는 피자를 좋아한다."),
]

df = pd.DataFrame(data, columns=["en", "ko"])
df

사용 디바이스: cuda


,en,ko
0,I am a student.,나는 학생이다.
1,He plays football.,그는 축구를 한다.
2,She likes coffee.,그녀는 커피를 좋아한다.
3,We study English.,우리는 영어를 공부한다.
4,They are friends.,그들은 친구이다.
5,I like pizza.,나는 피자를 좋아한다.


In [3]:
# 2. 토크나이저 직접 만들기 (영어/한국어)
# 영어: 소문자 + 특수문자 제거 + 공백 기준 나누기
def tokenize_en(text: str):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)   # 알파벳/숫자/공백만 남기기
    tokens = text.split()
    return tokens

# 한국어: 글자 단위로 단순 분리 (예: "나는 학생이다." -> ["나","는"," ","학","생","이","다","."])
def tokenize_ko(text: str):
    return list(text)

print(tokenize_en("He plays football."))
print(tokenize_ko("그는 축구를 한다."))

['he', 'plays', 'football']
['그', '는', ' ', '축', '구', '를', ' ', '한', '다', '.']


In [4]:
# Vocab(단어 사전) 직접 만들기
SPECIAL_TOKENS = ["<unk>", "<pad>", "<bos>", "<eos>"]

def build_vocab(texts, tokenizer):
    counter = Counter()
    for t in texts:
        counter.update(tokenizer(t))

    # 토큰들을 정렬해서 넣어도 되고, 순서대로 넣어도 괜찮음
    vocab = {}
    idx = 0
    for token in SPECIAL_TOKENS:
        vocab[token] = idx
        idx += 1

    for token in counter.keys():
        if token not in vocab:
            vocab[token] = idx
            idx += 1

    return vocab

vocab_en = build_vocab(df["en"], tokenize_en)
vocab_ko = build_vocab(df["ko"], tokenize_ko)

print("영어 vocab size:", len(vocab_en))
print("한글 vocab size:", len(vocab_ko))

# 인덱스 상수
PAD_IDX_EN = vocab_en["<pad>"]
PAD_IDX_KO = vocab_ko["<pad>"]
BOS_IDX_EN = vocab_en["<bos>"]
EOS_IDX_EN = vocab_en["<eos>"]
BOS_IDX_KO = vocab_ko["<bos>"]
EOS_IDX_KO = vocab_ko["<eos>"]

# 나중에 디코딩용 역매핑
rev_vocab_ko = {idx: token for token, idx in vocab_ko.items()}


영어 vocab size: 22
한글 vocab size: 32


In [5]:
# 문장을 숫자 시퀀스로 바꾸는 함수 (encode)
def encode_en(text, max_len=10):
    tokens = ["<bos>"] + tokenize_en(text) + ["<eos>"]
    ids = [vocab_en.get(t, vocab_en["<unk>"]) for t in tokens]

    if len(ids) < max_len:
        ids += [PAD_IDX_EN] * (max_len - len(ids))
    else:
        ids = ids[:max_len]

    return torch.tensor(ids, dtype=torch.long)


def encode_ko(text, max_len=12):
    tokens = ["<bos>"] + tokenize_ko(text) + ["<eos>"]
    ids = [vocab_ko.get(t, vocab_ko["<unk>"]) for t in tokens]

    if len(ids) < max_len:
        ids += [PAD_IDX_KO] * (max_len - len(ids))
    else:
        ids = ids[:max_len]

    return torch.tensor(ids, dtype=torch.long)

# 테스트
print(encode_en("I am a student."))
print(encode_ko("나는 학생이다."))

tensor([2, 4, 5, 6, 7, 3, 1, 1, 1, 1])
tensor([ 2,  4,  5,  6,  7,  8,  9, 10, 11,  3,  1,  1])


In [6]:
# Dataset & DataLoader 정의
class Seq2SeqDataset(Dataset):
    def __init__(self, df, max_len_en=10, max_len_ko=12):
        self.df = df
        self.max_len_en = max_len_en
        self.max_len_ko = max_len_ko

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        src_text = self.df.iloc[idx]["en"]
        trg_text = self.df.iloc[idx]["ko"]

        src_ids = encode_en(src_text, self.max_len_en)
        trg_ids = encode_ko(trg_text, self.max_len_ko)

        return src_ids, trg_ids

dataset = Seq2SeqDataset(df)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)

for src_batch, trg_batch in train_loader:
    print("src_batch shape:", src_batch.shape)  # [batch, src_len]
    print("trg_batch shape:", trg_batch.shape)  # [batch, trg_len]
    break

src_batch shape: torch.Size([2, 10])
trg_batch shape: torch.Size([2, 12])


In [7]:
# Encoder / Decoder / Seq2Seq 모델 정의
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=PAD_IDX_EN)
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True)

    def forward(self, src):
        # src: [batch, src_len]
        embedded = self.embedding(src)           # [batch, src_len, emb_dim]
        outputs, hidden = self.rnn(embedded)     # outputs: [batch, src_len, hidden_dim]
        return hidden                            # hidden: [1, batch, hidden_dim]


class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=PAD_IDX_KO)
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, input, hidden):
        # input: [batch] (현재 시점 토큰 인덱스)
        input = input.unsqueeze(1)               # [batch, 1]
        embedded = self.embedding(input)         # [batch, 1, emb_dim]
        output, hidden = self.rnn(embedded, hidden)   # output: [batch, 1, hidden_dim]
        prediction = self.fc_out(output.squeeze(1))   # [batch, output_dim]
        return prediction, hidden


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [batch, src_len]
        # trg: [batch, trg_len]
        batch_size = src.size(0)
        trg_len = trg.size(1)
        trg_vocab_size = self.decoder.fc_out.out_features

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        hidden = self.encoder(src)
        # 디코더 첫 입력: 타깃 시퀀스의 <bos>
        input = trg[:, 0]  # [batch]

        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[:, t, :] = output

            # 가장 높은 확률 토큰 선택
            top1 = output.argmax(1)

            # teacher forcing 여부
            use_tf = torch.rand(1).item() < teacher_forcing_ratio
            input = trg[:, t] if use_tf else top1

        return outputs


In [8]:
# 모델 생성 & 학습 루프
INPUT_DIM = len(vocab_en)
OUTPUT_DIM = len(vocab_ko)
EMB_DIM = 128
HID_DIM = 256

encoder = Encoder(INPUT_DIM, EMB_DIM, HID_DIM)
decoder = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM)
model = Seq2Seq(encoder, decoder, device).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX_KO)

print("학습 가능한 파라미터 수:",
      sum(p.numel() for p in model.parameters() if p.requires_grad))

학습 가능한 파라미터 수: 608032


In [11]:
def train_one_epoch(model, loader, optimizer, criterion, device, teacher_forcing_ratio=0.5):
    model.train()
    epoch_loss = 0.0

    for src, trg in loader:
        src = src.to(device)
        trg = trg.to(device)

        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio=teacher_forcing_ratio)
        # output: [batch, trg_len, output_dim]

        output_dim = output.size(-1)

        # <bos> 토큰을 제외하고 loss 계산
        output = output[:, 1:, :].reshape(-1, output_dim)  # [batch*(trg_len-1), output_dim]
        trg_y = trg[:, 1:].reshape(-1)                     # [batch*(trg_len-1)]

        loss = criterion(output, trg_y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(loader)


N_EPOCHS = 50

for epoch in range(1, N_EPOCHS + 1):
    loss = train_one_epoch(model, train_loader, optimizer, criterion, device,
                           teacher_forcing_ratio=0.5)
    print(f"[Epoch {epoch}/{N_EPOCHS}] train loss: {loss:.4f}")



[Epoch 1/50] train loss: 0.4491
[Epoch 2/50] train loss: 0.5150
[Epoch 3/50] train loss: 0.4019
[Epoch 4/50] train loss: 0.3340
[Epoch 5/50] train loss: 0.5879
[Epoch 6/50] train loss: 0.4286
[Epoch 7/50] train loss: 0.3526
[Epoch 8/50] train loss: 0.2668
[Epoch 9/50] train loss: 0.4562
[Epoch 10/50] train loss: 0.2005
[Epoch 11/50] train loss: 0.1806
[Epoch 12/50] train loss: 0.1623
[Epoch 13/50] train loss: 0.1426
[Epoch 14/50] train loss: 0.1263
[Epoch 15/50] train loss: 0.1123
[Epoch 16/50] train loss: 0.0984
[Epoch 17/50] train loss: 0.0911
[Epoch 18/50] train loss: 0.0816
[Epoch 19/50] train loss: 0.0728
[Epoch 20/50] train loss: 0.0663
[Epoch 21/50] train loss: 0.0603
[Epoch 22/50] train loss: 0.0552
[Epoch 23/50] train loss: 0.0508
[Epoch 24/50] train loss: 0.0472
[Epoch 25/50] train loss: 0.0437
[Epoch 26/50] train loss: 0.0404
[Epoch 27/50] train loss: 0.0380
[Epoch 28/50] train loss: 0.0359
[Epoch 29/50] train loss: 0.0339
[Epoch 30/50] train loss: 0.0320
[Epoch 31/50] train

In [12]:
# 번역함수 & 테스트
def translate_sentence(sentence, model, max_len_en=10, max_len_ko=12):
    model.eval()

    # 1) 입력 문장 인코딩
    src_ids = encode_en(sentence, max_len_en).unsqueeze(0).to(device)  # [1, src_len]

    # 2) 인코더
    with torch.no_grad():
        hidden = model.encoder(src_ids)

    # 3) 디코더 시작: <bos> 토큰
    input_token = torch.tensor([BOS_IDX_KO], dtype=torch.long, device=device)
    generated_ids = [BOS_IDX_KO]

    for _ in range(max_len_ko):
        with torch.no_grad():
            output, hidden = model.decoder(input_token, hidden)
            pred_token = output.argmax(1).item()

        generated_ids.append(pred_token)

        if pred_token == EOS_IDX_KO:
            break

        input_token = torch.tensor([pred_token], dtype=torch.long, device=device)

    # 4) 인덱스를 토큰(글자)로 변환
    tokens = [
        rev_vocab_ko[idx]
        for idx in generated_ids
        if idx in rev_vocab_ko and rev_vocab_ko[idx] not in ["<bos>", "<eos>", "<pad>"]
    ]

    return "".join(tokens)


# 데이터셋에 있는 문장들로 테스트
for i in range(len(df)):
    src_text = df.iloc[i]["en"]
    trg_text = df.iloc[i]["ko"]
    pred = translate_sentence(src_text, model)

    print(f"[원문] {src_text}")
    print(f"[정답] {trg_text}")
    print(f"[모델] {pred}")
    print("-" * 40)

# 새 문장 테스트
custom = "I like pizza."
print("=== 새 문장 테스트 ===")
print("입력:", custom)
print("출력:", translate_sentence(custom, model))

[원문] I am a student.
[정답] 나는 학생이다.
[모델] 나는 학생이다.
----------------------------------------
[원문] He plays football.
[정답] 그는 축구를 한다.
[모델] 그는 축구를 한다.
----------------------------------------
[원문] She likes coffee.
[정답] 그녀는 커피를 좋아한다.
[모델] 그녀는 커피를 좋아한다
----------------------------------------
[원문] We study English.
[정답] 우리는 영어를 공부한다.
[모델] 우리는 영어를 공부한다
----------------------------------------
[원문] They are friends.
[정답] 그들은 친구이다.
[모델] 그들은 친구이다.
----------------------------------------
[원문] I like pizza.
[정답] 나는 피자를 좋아한다.
[모델] 나는 피자를 좋아한다.
----------------------------------------
=== 새 문장 테스트 ===
입력: I like pizza.
출력: 나는 피자를 좋아한다.
